This is the basic approach of sentiment analysis using LSTM

The data used is from linguipedia-codefest-natural-language-processing-1 contest in analytics Vidya

Tokenizer and Word embeddings used are Keras's

Accuracy(with this code) - 88% on test data

#### Import libraries

In [1]:
import os
import pandas as pd
import numpy as np
from keras.layers.merge import concatenate
from keras.layers import Dense, Input, Dropout, Bidirectional, LSTM, Embedding
from keras.models import Model
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle
import re
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import statistics
import string
import matplotlib.pyplot as pp

Using TensorFlow backend.
/home/saialekhyar/anaconda3/envs/py35/lib/python3.5/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.23) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


#### Read data

In [2]:
path = os.getcwd()

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'LSTM_SentimentAnalysis.ipynb',
 'test_preds.csv',
 'sample_submission_LnhVWA4.csv',
 'test_oJQbWVk.csv',
 'train_2kmZucJ.csv',
 'submission_LSTM_1.csv']

In [4]:
df = pd.read_csv('train_2kmZucJ.csv')

In [5]:
df.head(10)

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...
5,6,1,iPhone software update fucked up my phone big ...
6,7,0,Happy for us .. #instapic #instadaily #us #son...
7,8,0,New Type C charger cable #UK http://www.ebay.c...
8,9,0,Bout to go shopping again listening to music #...
9,10,0,Photo: #fun #selfie #pool #water #sony #camera...


In [6]:
print('Shape of the data is ', df.shape)

Shape of the data is  (7920, 3)


In [7]:
df.label.value_counts(normalize=True)

0    0.744192
1    0.255808
Name: label, dtype: float64

The data distribution is 75%-25% which is not extremely biased data

#### Identify Null values

In [8]:
df.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

#### Data Preprocessing

In [9]:
tweets_train = list(df['tweet'])

In [10]:
REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_tweets(tweets):
    tweets = [REPLACE_NO_SPACE.sub("", line.lower()) for line in tweets]
    tweets = [REPLACE_WITH_SPACE.sub(" ", line) for line in tweets]
    return tweets

tweets_train_clean = preprocess_tweets(tweets_train)

In [15]:
test_df = pd.read_csv('test_oJQbWVk.csv')
tweets_test = list(test_df['tweet'])
tweets_test_clean = preprocess_tweets(tweets_test)
# no_stop_words_test = remove_stop_words(tweets_test_clean)
# lemmatized_tweets_test = get_s(no_stop_words_test)
tweets_test_clean_1 = [clean_sentence(item) for item in tweets_test_clean]

In [11]:
special_chars = list(string.punctuation)

In [12]:
def clean_sentence(sent):
    for item in special_chars:
        sent = sent.replace(item, ' '+ item + ' ')
    return re.sub(' +', ' ', sent)

In [16]:
len_list = [len(clean_sentence(item).split()) for item in tweets_train_clean]

In [17]:
print('Max length of tweets is ', max(len_list))

Max length of tweets is  89


In [18]:
statistics.median(len_list)

24.0

In [19]:
max_len =  max(len_list)#int(statistics.median(len_list))

In [20]:
tweets_train_clean_1 = [clean_sentence(item) for item in tweets_train_clean]

In [22]:
#max(map(len, sentences))

In [23]:
print('preprocessing data...')
tokenizer = Tokenizer()#num_words= vocabulary_size

tokenizer.fit_on_texts(tweets_train_clean_1+tweets_test_clean_1)#fitted on both test and train
vocab_size = len(tokenizer.word_index)+1
sentences = tokenizer.texts_to_sequences(tweets_train_clean_1)
X_sentences = pad_sequences(sentences, maxlen=max_len, padding='post', truncating = 'post')

# X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1,shuffle=False)

preprocessing data...


In [24]:
y = df['label'].astype(str)

In [25]:
le = preprocessing.LabelEncoder()
le.fit(y)
y = to_categorical(le.transform(y), num_classes=None)
list(le.classes_)

['0', '1']

In [26]:
print('Shape of tweets tensor:', X_sentences.shape)
print('Shape of label tensor:', y.shape)
num_classes = len(le.classes_)
print('num_classes : ', num_classes)

Shape of tweets tensor: (7920, 89)
Shape of label tensor: (7920, 2)
num_classes :  2


In [27]:
#X_train, X_test, y_train, y_test = train_test_split(X_sentences, y, test_size=0.1, random_state=0)

In [28]:
import keras.backend as K
def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [43]:
max_words = max_len 
epochs = 12
max_features = vocab_size#20000
batch_size = 1024 # batch size
dims = 100

# SENTENCES input branch 
sent_inputs = Input(shape=(max_words,))
branch_1 = Embedding(max_features, dims, input_length=max_len)(sent_inputs)

joint= branch_1
joint = Bidirectional(LSTM(128))(joint)
# joint = Dropout(0.5)(joint)
predictions = Dense(num_classes, activation='softmax')(joint)

model = Model(inputs = sent_inputs, outputs=[predictions])#[sent_inputs, tags_inputs]

model.compile(loss='categorical_crossentropy',
                   optimizer='rmsprop',
                   metrics=['accuracy', f1_score])

In [44]:
history = model.fit(X_sentences, y, epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)

Epoch 1/15
7920/7920 [==============================] - 34s 4ms/step - loss: 0.6814 - acc: 0.7340 - f1_score: 0.7340
Epoch 2/15
7920/7920 [==============================] - 31s 4ms/step - loss: 0.4982 - acc: 0.7442 - f1_score: 0.7442
Epoch 3/15
7920/7920 [==============================] - 34s 4ms/step - loss: 0.4289 - acc: 0.8172 - f1_score: 0.8172
Epoch 4/15
7920/7920 [==============================] - 33s 4ms/step - loss: 0.2748 - acc: 0.9057 - f1_score: 0.9057
Epoch 5/15
7920/7920 [==============================] - 33s 4ms/step - loss: 0.1967 - acc: 0.9239 - f1_score: 0.9239
Epoch 6/15
7920/7920 [==============================] - 31s 4ms/step - loss: 0.1705 - acc: 0.9356 - f1_score: 0.9356
Epoch 7/15
7920/7920 [==============================] - 31s 4ms/step - loss: 0.1411 - acc: 0.9461 - f1_score: 0.9461
Epoch 8/15
7920/7920 [==============================] - 31s 4ms/step - loss: 0.1138 - acc: 0.9573 - f1_score: 0.9573
Epoch 9/15
7920/7920 [==============================] - 31s 4ms/

#### Read test data

In [45]:
test_df = pd.read_csv('test_oJQbWVk.csv')
tweets_test = list(test_df['tweet'])
tweets_test_clean = preprocess_tweets(tweets_test)
# no_stop_words_test = remove_stop_words(tweets_test_clean)
# lemmatized_tweets_test = get_s(no_stop_words_test)
tweets_test_clean_1 = [clean_sentence(item) for item in tweets_test_clean]

In [46]:
test_df.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [47]:
test_df.shape

(1953, 2)

In [48]:
tweets_test = list(test_df['tweet'])
tweets_test_clean = preprocess_tweets(tweets_test)
# no_stop_words_test = remove_stop_words(tweets_test_clean)
# lemmatized_tweets_test = get_s(no_stop_words_test)
tweets_test_clean_1 = [clean_sentence(item) for item in tweets_test_clean]

In [49]:
sentences_test = tokenizer.texts_to_sequences(tweets_test_clean_1)
X_sentences_test = pad_sequences(sentences_test, maxlen=max_len, padding='post', truncating = 'post')

In [50]:
preds = model.predict(X_sentences_test)

In [51]:
label_list = [le.inverse_transform([np.argmax(pred, axis = -1)])[0] for pred in preds]

/home/saialekhyar/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/saialekhyar/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/saialekhyar/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/saialekhyar/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151

/home/saialekhyar/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/saialekhyar/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/saialekhyar/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/saialekhyar/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151

In [52]:
test_df['label'] = label_list

In [53]:
test_df.label.value_counts()

0    1452
1     501
Name: label, dtype: int64

In [54]:
test_df.columns

Index(['id', 'tweet', 'label'], dtype='object')

In [55]:
final_df = test_df[['id', 'label']]

In [56]:
final_df.to_csv('submission_LSTM_3.csv', index= None)

In [86]:
test_df.to_csv('test_preds.csv')